In [1]:
import pandas as pd
import numpy as np

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import seaborn as sns

## Read data

In [2]:
file_name = "../File/FrontendFileGroup/storm-frontend-20200307-group.txt"
# file_name = "./storm-frontend-20200307-mask.txt"

raw_data = pd.read_csv(file_name, index_col= 0, nrows = 1e4)

C:\Users\Fabri\AppData\Local\Temp\ipykernel_5984\2175162335.py:4: DtypeWarning: Columns (144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(file_name, index_col= 0, nrows = 1e4)


In [3]:
# raw_data = raw_data.drop(raw_data.index[66], axis=0)

In [4]:
raw_data.msg180.notna()

274b7155-f4bc-4649-a0ff-946717c6ea92    False
1281ffc9-b3a9-4a6d-af84-02030bc4b4bb    False
f4ecc144-881b-4eb4-a456-7563e5bb5209    False
ad7c6908-736d-4af0-8c37-e2d4d20b2936    False
93fc8683-b4bb-48e3-bbb9-0ffe24c8d81b    False
                                        ...  
2b4fdb6f-e999-4405-8766-73c06344598a    False
e189c148-feb7-43b3-9694-a1c4677c2457    False
9d433c1f-0280-4f1a-9ffe-6591ace3517f    False
9988999c-73ae-4d09-a082-650698173328    False
1fd42fa2-7c7d-4a04-844f-838ff17f893a    False
Name: msg180, Length: 10000, dtype: bool

In [5]:
errors = pd.DataFrame()

for i in raw_data.index:
    message = str()
    for msg in raw_data.loc[i][4:].dropna():
        message += ' ' + str(msg)
    raw_data['msg1'][i] = message
        #         raw_data.loc[i]['msg1'] += ' ' + msg
# for i in raw_data.index:
#     lista = raw_data.loc[i]
#     idx = lista.pop('id')
#     temp = pd.DataFrame(dict(lista), index=[idx], copy=True)
#     errors = errors.append(temp)

C:\Users\Fabri\AppData\Local\Temp\ipykernel_5984\148899882.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data['msg1'][i] = message


KeyboardInterrupt: 

In [ ]:
errors = raw_data.rename(columns={'msg1':'message'}).drop(raw_data.columns[5:], axis= 1)
errors

In [ ]:
print("Total number of errors:", errors.shape[0])

## Extract information
Once we read the data, we can focus on the variable message and try to extract meaningful information from it.

### Clearing and tokenizing messages

In [ ]:
# Reduce to lowercase and split in tokens
tokens_per_message = [x.lower().split() for x in errors.message]

In [ ]:
# Retrieve the set of all tokens used in the error messages
word_set = set()
for mess in tokens_per_message:
    word_set = word_set.union(set(mess))
    
word_set

In [ ]:
print("We have {} error messages, for a total of {} unique tokens adopted.".format(
    len(tokens_per_message), len(word_set)))

In [ ]:
# Setup a dictionary with frequency of all tokens per each message (initialized to 0)
word_dict = [dict.fromkeys(word_set, 0) for i in range(len(tokens_per_message))]
print("Number of tokens:", len(word_dict[0]))

word_dict[0]

In [ ]:
# Compute raw frequencies of each token per each message
for i in range(len(errors.message)):
    for word in tokens_per_message[i]:
        word_dict[i][word] += 1
        
word_dict[0]

In [ ]:
print("Dictionary dimensions:\n", pd.DataFrame(word_dict).shape)

# Visualization
pd.DataFrame(word_dict)

### Compute TF-IDF scores

Once we have messages divided in words and we have computed the raw frequencies of each token in each sentence, then we can proceed and compute the **tf-idf** score for each message.

In [ ]:
c = 0
for i, dic in enumerate(tokens_per_message):
    if not len(dic):
        print(i, errors.loc[i])
        c += 1

In [ ]:
print("Warning: there are {} blanck messages which will be excluded from the analysis.".format(c))

Frequenza delle parole in uno stesso processo

In [ ]:
def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        try:
            tf[word] = count/sum_nk
        except ZeroDivisionError:
            tf[word] = 0
    return tf

tf = [compute_tf(word_dict[i], tokens_per_message[i])
      for i in range(len(tokens_per_message))] #if sum(word_dict[i].values())]
# tf_A = compute_tf(word_dict_A, l_A)
# tf_B = compute_tf(word_dict_B, l_B)
# tf_C = compute_tf(word_dict_C, l_C)

Frequenza con cui una parola compare nei processi

In [ ]:
def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1

    for word, v in idf.items():
        idf[word] = np.log(n / float(v))
    return idf

idf = compute_idf(word_dict)
# idf = compute_idf([word_dict_A, word_dict_B, word_dict_C])

In [ ]:
len(word_dict), len(tf), len(tf[0]), len(idf)

In [ ]:
def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf
    
tf_idf =  [compute_tf_idf(tf[i], idf) for i in range(len(tf))]
# tf_idf_A = compute_tf_idf(tf_A, idf)
# tf_idf_B = compute_tf_idf(tf_B, idf)
# tf_idf_C = compute_tf_idf(tf_C, idf)

In [ ]:
pd.DataFrame(tf_idf)

### Alternative using Scikit-Learn: TfidfVectorizer

In [ ]:
%%time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd


print("Extracting features from the training dataset using a sparse vectorizer")
# t0 = time()
vectorizer = TfidfVectorizer(max_df=0.8, min_df=0.02, stop_words='english',
                             use_idf=True)
# vectorizer = TfidfVectorizer(stop_words='english',
#                              use_idf=True)
X = vectorizer.fit_transform(errors.message)

In [ ]:
vectorizer.get_params()

# Clustering

As a first attempt we can try a simple clustering algorithm such as *K-means*. 

<div class="alert alert-block alert-info">
<b>Note:</b> We repeat the information extraction step through the scikit-learn routine TfidfVectorizer just for convenience (so to avoid problems with variable types and to apply easily the clustering algorithm).
</div>

In [ ]:
# import libraries
from __future__ import print_function

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time

import numpy as np

# Extract TF-IDF information
print("Extracting features from the training dataset using a sparse vectorizer")
t0 = time()
vectorizer = TfidfVectorizer(max_df=0.8, min_df=0.02, stop_words='english',
                             use_idf=True)
# vectorizer = TfidfVectorizer(stop_words='english',
#                              use_idf=True)
X = vectorizer.fit_transform(errors.message)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()


# Apply LSA for dimensionality reduction to get a lower-dimensional embedding space
print("Performing dimensionality reduction using LSA")
t0 = time()

# Vectorizer results are normalized, which makes KMeans behave as
# spherical k-means for better results. Since LSA/SVD results are
# not normalized, we have to redo the normalization.
svd = TruncatedSVD(25)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(X)

print("done in %fs" % (time() - t0))

explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(
    int(explained_variance * 100)))

print()

Once we have a somewhat convenient subspace for representing the words/tokens, and hence the messages, then we cann proceed and apply the clustering algorithm.

<div class="alert alert-block alert-info">
    <b>Note:</b> In the following, we pretend the variable <i>category</i> as a true label for the clustering results. This is just to show how one could evaluate results of the unsupervised model when target labels are available.
</div>

In [ ]:
%%time

# set number of clusters (hyperparameter)
n_clusters = 10

# run K-Means algorithm: 6 clusters
km = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=500, n_init=100,
            verbose=1)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

## Results
Show graphs and stats here

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('_mpl-gallery')

label = np.unique(km.labels_)
count = [Counter(km.labels_)[i] for i in label]

# plot:
fig, ax = plt.subplots(figsize = (6, 4))

for l, c in zip(label, count):
    ax.barh(l, c, linewidth=0.5, edgecolor="white", label=errors.message[errors.kmean_labels==l][0])

ax.set(xticks=np.arange(0, max(count), 10), yticks=label)

# ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
#        ylim=(0, 56), yticks=np.linspace(0, 56, 9))

# ax.legend()
plt.show()

In [ ]:
for l in label:
    print(errors.message[errors.kmean_labels==l][0], '\n')

First thing we can look at are the centroids. 

In [ ]:
print("We have {} centroids represented as {}-dimensional points.".format(km.cluster_centers_.shape[0], km.cluster_centers_.shape[1]))
print("Let see the first one for example:")

km.cluster_centers_[0]

Observations labels, instead, can be accessed through the attribute **labels_** of the clustering object:

In [ ]:
from collections import Counter, defaultdict

# print the numerosity of each cluster
print(Counter(km.labels_))

Now we can look for example to the messages that fall into the same class:

In [ ]:
errors["kmean_labels"] = km.labels_

for msg in errors.message[errors.kmean_labels==8]:
    print(msg, '\n')

### Check the true label (if available)

Questa parte servirebbe se sapessi a priori a quale categoria appartiene ogni processo per controllare che l'algoritmo abbia prodotto i risultati attesi, ma non è questo il caso

## Conclusions and Next Steps
Summarize findings here